Before we get into creating and deploying this automated model we need to take a look at the scope of the project and all of the parts involved. This is important not only because it will help us to keep organized, but also because without one of these moving parts the whole system is rendered useless. These are the different things that we will need to do to get this model up and running:
### Building The Model
- Gather initial training data from an endpoint.
- Train a model off of the initial training data, and ideally get relatively decent accuracy.
- Serialize the model.

### Building An Endpoint
- Create an endpoint to return predictions from the serialized model.
- Create a script to pull new incoming data, as well as read-in old data and combine the two (not necessary for -   LSTM models.)

### Deployment
- Setup a web-server.
- Setup a supervisor
- Setup a cronjob to run the data script automatically.

# Buidling The Model

In [1]:
import requests
data = requests.get("https://api.covid19api.com/world")
# https://api.covid19api.com/summary

In [2]:
data.text

'[\n  {\n    "NewConfirmed": 151812,\n    "TotalConfirmed": 3503238,\n    "NewDeaths": 8547,\n    "TotalDeaths": 247349,\n    "NewRecovered": 87183,\n    "TotalRecovered": 1125094\n  },\n  {\n    "NewConfirmed": 80678,\n    "TotalConfirmed": 4791229,\n    "NewDeaths": 3507,\n    "TotalDeaths": 321071,\n    "NewRecovered": 40774,\n    "TotalRecovered": 1733320\n  },\n  {\n    "NewConfirmed": 95756,\n    "TotalConfirmed": 5677213,\n    "NewDeaths": 4319,\n    "TotalDeaths": 357189,\n    "NewRecovered": 55218,\n    "TotalRecovered": 2286305\n  },\n  {\n    "NewConfirmed": 70272,\n    "TotalConfirmed": 3039914,\n    "NewDeaths": 4620,\n    "TotalDeaths": 211113,\n    "NewRecovered": 28234,\n    "TotalRecovered": 893322\n  },\n  {\n    "NewConfirmed": 0,\n    "TotalConfirmed": 3842902,\n    "NewDeaths": 0,\n    "TotalDeaths": 270014,\n    "NewRecovered": 0,\n    "TotalRecovered": 1233826\n  },\n  {\n    "NewConfirmed": 101362,\n    "TotalConfirmed": 4617583,\n    "NewDeaths": 5335,\n    "To

In [3]:
import pandas as pd
df = pd.read_json(data.text)

In [4]:
df.head(5)

,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered
0,151812,3503238,8547,247349,87183,1125094
1,80678,4791229,3507,321071,40774,1733320
2,95756,5677213,4319,357189,55218,2286305
3,70272,3039914,4620,211113,28234,893322
4,0,3842902,0,270014,0,1233826


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df)

In [6]:
feature = 'TotalConfirmed'
target = 'TotalDeaths'
trainX = train[feature]
trainy = train[target]
testX = test[feature]
testy = test[target]

In [7]:
from sklearn.linear_model import Lasso
model = Lasso()

In [8]:
import numpy as np

In [9]:
trainX = trainX.to_numpy().reshape(-1,1)
trainy = trainy.to_numpy().reshape(-1,1)

In [10]:
model.fit(trainX,trainy)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [11]:
testX = testX.to_numpy().reshape(-1,1)
yhat = model.predict(testX)

In [12]:
print(yhat)

[208886.28736224 173514.512863   166244.70075448 223663.71886688
 395076.94483021 301565.52962188 342845.14013211 238187.41603695
 361132.9904273  284227.62699057 243043.21262318 233564.66693118
 366227.50964866]


In [13]:
from sklearn.metrics import r2_score
print(r2_score(testy,yhat))

0.9724050814300688


In [14]:
import pickle as pkl

In [18]:
pkl.dump(model,open( "model.pkl", "wb" ))